In [ ]:
import allel
from allel.stats import fst, diversity
import pandas as pd
import numpy as np
import subprocess
import itertools

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def filter_variants_for_fst(genotype, subpops: dict) -> allel.GenotypeArray:
    pop1, pop2 = list(subpops.keys())
    acs = genotype.count_alleles_subpops(subpops)
    acu = allel.AlleleCountsArray(acs[pop1][:] + acs[pop2][:])
    
    flt = acu.is_segregating() & (acu.max_allele() == 1)
    print('retaining', np.count_nonzero(flt), 'SNPs')
    
    filtered_genotype = genotype.compress(flt, axis=0)

    return filtered_genotype

def dxy(genotype, subpops):
    pop1, pop2 = list(subpops.keys())
    acs = genotype.count_alleles_subpops(subpops)
    pairwise_diff = diversity.mean_pairwise_difference_between(acs[pop1], acs[pop2])

    return pairwise_diff


Extract samples from Mexican populations, excluding cross-contaminated samples

In [ ]:
pops_file = "../data/samples/populations.txt"
admixed = "../data/var/admixture/admixed_individuals.csv"
mex = ["Amanitaspjack3", "Amanitaspjack5", "Amanitaspjack6"]
pops = pd.read_table(pops_file, header=None, names=["sample", "population"])
admixed = pd.read_csv(admixed)
pops_mex = pops[pops["population"].isin(mex) & ~pops["sample"].isin(admixed["sample"])]
pops_mex

In [ ]:
csv_samples = ",".join(pops_mex["sample"].tolist())
csv_samples

Create a VCF file exclusive to Mexican samples

In [ ]:
vcf_file = "../data/var/merged_variants.vcf.gz"
output_file = "../data/var/mex_variants.vcf.gz"

cmd_subset_samples = ["bcftools", "view", "-s", f"{csv_samples}", vcf_file]
cmd_remove_info = ["bcftools", "annotate", "--remove", "INFO"]
cmd_refill_tags = ["bcftools", "+fill-tags", "--", "-t", "all"]
cmd_keep_full_callset = [
    "bcftools", "view", 
    "-i", "INFO/F_MISSING = 0 & AF > 0 & AF < 1",
    "-m", "2", 
    "-M", "2", 
    "-v", "snps", 
    "-Oz", 
    "-o", output_file,
    "--write-index",
]

process1 = subprocess.Popen(cmd_subset_samples, stdout=subprocess.PIPE)
process2 = subprocess.Popen(cmd_remove_info, stdin=process1.stdout, stdout=subprocess.PIPE)
process1.stdout.close() # Allow process1 to receive a SIGPIPE if process2 exits
process3 = subprocess.Popen(cmd_refill_tags, stdin=process2.stdout, stdout=subprocess.PIPE)
process2.stdout.close()
process4 = subprocess.Popen(cmd_keep_full_callset, stdin=process3.stdout)
process3.stdout.close()
# wait for the last process to finish
process4.communicate()

In [ ]:
callset = allel.read_vcf(output_file)
samples = callset['samples']
gt = allel.GenotypeArray(callset['calldata/GT'])

In [ ]:
pop_indices = {
    pop: np.where(np.isin(samples, pops_mex[pops_mex["population"] == pop]["sample"]))[0]
    for pop in pops_mex["population"].unique()
}
pop_indices

In [ ]:
pop_combo = list(itertools.combinations(pops_mex["population"].unique(), 2))
pop_combo

In [ ]:
fsts_df = pd.DataFrame()
for pop1, pop2 in pop_combo:
    subpops_idx = {pop: pop_indices[pop] for pop in (pop1, pop2)}
    flt_gt = filter_variants_for_fst(gt, subpops_idx)
    acs = flt_gt.count_alleles_subpops(subpops_idx)

    a, b, c = fst.weir_cockerham_fst(
        flt_gt, 
        subpops=[subpops_idx[pop1], 
                 subpops_idx[pop2]], 
        max_allele=1,
        blen=1000,
    )
    
    with np.errstate(divide='ignore', invalid='ignore'):
        fst_per_variant = (a / (a + b + c))[:, 0]
    fst_value = np.nanmean([val for val in fst_per_variant if not np.isnan(val)])
    fst_global = (sum(a) / sum(a + b + c))[0]
    
    num, den = fst.hudson_fst(acs[pop1], acs[pop2])
    snp_fst_hudson = num / den
    global_fst_hudson = sum(num) / sum(den)
    fst_value_hudson = np.nanmean([val for val in snp_fst_hudson if not np.isnan(val)])
    
    print(f"Mean Weir-Cockerham's Fst between {pop1} and {pop2}: {fst_value:.4f}")
    print(f"Global Weir-Cockerham's Fst between {pop1} and {pop2}: {fst_global:.4f}")
    print(f"Mean Hudson's Fst between {pop1} and {pop2}: {fst_value_hudson:.4f}")
    print(f"Global Hudson's Fst between {pop1} and {pop2}: {global_fst_hudson:.4f}")
    df = pd.DataFrame({
        "fst_per_variant_wc": fst_per_variant,
        "fst_per_variant_hudson": snp_fst_hudson,
    })
    df["fst_global_wc"] = fst_global
    df["fst_global_hudson"] = global_fst_hudson
    df["populations"] = f"{pop1} vs {pop2}"

    fsts_df = pd.concat([fsts_df, df], ignore_index=True)

In [ ]:
sns.boxplot(data=fsts_df, x="populations", y="fst_per_variant_wc")
plt.ylabel("Fst per variant (Weir-Cockerham)")
plt.xlabel("Population Pair")
plt.title("Per-variant Weir-Cockerham Fst distribution")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
sns.boxplot(data=fsts_df, x="populations", y="fst_per_variant_hudson")
plt.ylabel("Fst per variant (Hudson)")
plt.xlabel("Population Pair")
plt.title("Per-variant Hudson's Fst distribution")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
dxy_df = pd.DataFrame()
for pop_pair in pop_combo:
    subpops_idx = {pop: pop_indices[pop] for pop in pop_pair}
    per_site_diff = dxy(gt, subpops_idx)
    print(f"{pop_pair}: {np.mean(per_site_diff)}")
    dxy_df = pd.concat((dxy_df, pd.DataFrame({
        "dxy": per_site_diff,
        "populations": f"{pop_pair[0]} vs {pop_pair[1]}",
    })), ignore_index=True)

In [ ]:
sns.boxplot(data=dxy_df, x="populations", y="dxy")
plt.ylabel("Dxy per variant")
plt.xlabel("Population Pair")
plt.title("Per-variant Dxy distribution")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()